# [【SOTA】マイナビ × SIGNATE Student Cup 2019: 賃貸物件の家賃予測](https://signate.jp/competitions/264)

## データ読み込み

In [34]:
import pandas as pd
import numpy as np
import pathlib
import os

# 学習データ、テストデータの読み込み
train_path = pathlib.Path("./DATA/train.csv")
test_path = pathlib.Path("./DATA/test.csv")

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [472]:
test_data.head()

,id,所在地,アクセス,間取り,築年数,方角,面積,所在階,バス・トイレ,キッチン,放送・通信,室内設備,駐車場,周辺環境,建物構造,契約期間,予測結果
0,31471,東京都世田谷区深沢５丁目1-27,東急大井町線\t等々力駅\t徒歩15分\t\t東急田園都市線\t桜新町駅\t徒歩24分\t\...,2LDK,49年0ヶ月,南,50.22m2,8階／8階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t温水洗浄便座／...,ガスコンロ／\tコンロ3口／\tシステムキッチン\t／\t給湯／\tL字キッチン,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\tエレベーター\t公営水...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【コンビニ】 74m\t【スーパー】 215m\t【病院】 313m\t【小学校】 219m...,鉄骨造,NaN,158098
1,31472,東京都目黒区八雲１丁目11-8,東急東横線\t都立大学駅\t徒歩6分\t\t東急東横線\t自由が丘駅\t徒歩17分\t\t東...,1R,0年2ヶ月,南東,20.88m2,3階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCSアンテナ／\tBSアンテナ／\tインターネット使用料無料,エアコン付\tシューズボックス／\tフローリング／\t室内洗濯機置場\t公営水道／\t下水／...,"駐車場\t近隣\t34,000円(税込)\t距離200m\t駐輪場\t無\tバイク置き場\t無",【スーパー】 128m\t【スーパー】 313m\t【コンビニ】 374m\t【コンビニ】 ...,RC（鉄筋コンクリート）,2年間,80236
2,31473,東京都豊島区池袋本町２丁目22-2,東武東上線\t北池袋駅\t徒歩10分\t\t埼京線\t板橋駅\t徒歩16分\t\t山手線\t...,1K,23年4ヶ月,南東,26.93m2,1階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座,ガスコンロ／\t給湯,インターネット対応／\t光ファイバー／\tCATV,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t2面採光／\t室内洗濯...,"駐車場\t空有\t5,000円(税込)\t(敷金あり\t1ヶ月)\t駐輪場\t空有\tバイク...",【スーパー】 529m\t【スーパー】 810m\t【コンビニ】 316m\t【コンビニ】 ...,鉄骨造,2年間,94671
3,31474,東京都杉並区和泉１丁目,丸ノ内線(中野坂上－方南町)\t方南町駅\t徒歩10分\t\t京王線\t代田橋駅\t徒歩8分...,1K,36年2ヶ月,南東,23.57m2,1階／2階建,シャワー,ガスコンロ／\t給湯,NaN,エアコン付\tシューズボックス／\t2面採光／\t室外洗濯機置場\t公営水道／\t下水／\t...,駐輪場\t空有\t0円,【スーパー】 286m\t【スーパー】 702m\t【コンビニ】 489m\t【コンビニ】 ...,木造,2年間,69022
4,31475,東京都杉並区堀ノ内２丁目,丸ノ内線(中野坂上－方南町)\t方南町駅\t徒歩12分\t\t丸ノ内線(池袋－荻窪)\t新高...,2LDK,45年4ヶ月,南,50m2,4階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別,コンロ設置可（口数不明）／\t給湯,インターネット対応,冷房／\tエアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置...,"駐車場\t近隣\t22,600円\t距離177m",【コンビニ】 312m\t【コンビニ】 436m\t【スーパー】 902m\t【コンビニ】 ...,RC（鉄筋コンクリート）,2年間,133378


## 前処理1

In [35]:
# 簡単な前処理
train_data_1 = train_data[["id", "賃料", "間取り", "面積", "所在階"]]
test_data_1 = test_data[["id", "間取り", "面積", "所在階"]]

In [36]:
# 間取りにlabel encodingを適用
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(pd.concat([train_data_1["間取り"],test_data_1["間取り"]]))

train_data_1["間取りID"] = le.transform(train_data_1["間取り"])
test_data_1["間取りID"] = le.transform(test_data_1["間取り"])

<ipython-input-36-95a8caebc059>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_1["間取りID"] = le.transform(train_data_1["間取り"])
<ipython-input-36-95a8caebc059>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_1["間取りID"] = le.transform(test_data_1["間取り"])


In [37]:
# 面積のカラムタイトルを面積[m2]に置換
train_data_1 = train_data_1.rename(columns={"面積": "面積m2"})
test_data_1 = test_data_1.rename(columns={"面積": "面積m2"})

In [38]:
# カラムタイトル面積[m2]の要素からm2を削除
train_data_1['面積m2'] = train_data_1['面積m2'].str.replace('m2', '').astype(float)
test_data_1['面積m2'] = test_data_1['面積m2'].str.replace('m2', '').astype(float)

In [39]:
# 所在階の中身を"／"で2つの列に分割
train_data_1 = pd.concat([train_data_1, train_data_1['所在階'].str.split('／', expand=True)], axis=1)
test_data_1 = pd.concat([test_data_1, test_data_1['所在階'].str.split('／', expand=True)], axis=1)

In [40]:
# 分割した所在階のカラム名変更　0:所在、1:階層
train_data_1 = train_data_1.rename(columns={0:"所在", 1:"階層"})
test_data_1 = test_data_1.rename(columns={0:"所在", 1:"階層"})

In [41]:
# 所在と階層の要素を整形
train_data_1['所在'] = train_data_1['所在'].str.replace('階', '')
train_data_1['所在'] = train_data_1['所在'].str.replace('建', '')
train_data_1['所在'] = train_data_1['所在'].str.replace('地下', '-')
train_data_1['所在'] = train_data_1['所在'].str.replace('\（.*\）', '', regex=True)
train_data_1['階層'] = train_data_1['階層'].str.replace('階建', '')
train_data_1['階層'] = train_data_1['階層'].str.replace('\（.*\）', '', regex=True)
train_data_1['所在'] = train_data_1['所在'].replace('', np.nan)

test_data_1['所在'] = test_data_1['所在'].str.replace('階', '')
test_data_1['所在'] = test_data_1['所在'].str.replace('建', '')
test_data_1['所在'] = test_data_1['所在'].str.replace('地下', '-')
test_data_1['所在'] = test_data_1['所在'].str.replace('\（.*\）', '', regex=True)
test_data_1['階層'] = test_data_1['階層'].str.replace('階建', '')
test_data_1['階層'] = test_data_1['階層'].str.replace('\（.*\）', '', regex=True)
test_data_1['所在'] = test_data_1['所在'].replace('', np.nan)

In [42]:
# "階層"がNoneの箇所を"所在"の値で埋める
train_data_1['階層'].fillna(train_data_1['所在'], inplace=True)
test_data_1['階層'].fillna(test_data_1['所在'], inplace=True)

In [43]:
# 所在と階層の要素をfloat型に変換する
train_data_1['所在'] = train_data_1['所在'].astype(float)
train_data_1['階層'] = train_data_1['階層'].astype(float)

test_data_1['所在'] = test_data_1['所在'].astype(float)
test_data_1['階層'] = test_data_1['階層'].astype(float)

In [44]:
# 間取りと所在階のカラムを削除
train_data_1 = train_data_1.drop('間取り', axis=1)
train_data_1 = train_data_1.drop('所在階', axis=1)

test_data_1 = test_data_1.drop('間取り', axis=1)
test_data_1 = test_data_1.drop('所在階', axis=1)

In [45]:
# IDの削除
train_data_1_no_ID = train_data_1.drop(["id"], axis=1)
test_data_1_no_ID = test_data_1.drop(["id"], axis=1)

## データチェック1

In [46]:
train_data_1.head()

,id,賃料,面積m2,間取りID,所在,階層
0,1,75000,20.01,3,1.0,12.0
1,2,76000,16.50,9,5.0,10.0
2,3,110000,22.05,3,12.0,15.0
3,4,150000,60.48,14,3.0,4.0
4,5,74000,39.66,10,1.0,2.0


In [47]:
test_data_1.head()

,id,面積m2,間取りID,所在,階層
0,31471,50.22,14,8.0,8.0
1,31472,20.88,9,3.0,4.0
2,31473,26.93,3,1.0,4.0
3,31474,23.57,3,1.0,2.0
4,31475,50.00,14,4.0,4.0


In [48]:
train_data_1.isnull().sum()

id         0
賃料         0
面積m2       0
間取りID      0
所在       120
階層         0
dtype: int64

In [49]:
test_data_1.isnull().sum()

id         0
面積m2       0
間取りID      0
所在       114
階層         1
dtype: int64

## 前処理2

### 訓練データ

In [30]:
# 埋める前の確認
print(train_data_1[train_data_1["所在"].isnull()])

          id      賃料    面積m2  間取りID  所在    階層
99       100  150500   68.83     14 NaN   3.0
122      123   58000   18.98      9 NaN   8.0
583      584  145000   60.72     17 NaN  12.0
671      672  900000  231.52     31 NaN   3.0
814      815  145000   71.36     21 NaN   2.0
...      ...     ...     ...    ...  ..   ...
29667  29668  127000   35.03      5 NaN   3.0
29800  29801   92000   50.94      6 NaN   2.0
30306  30307  153000   50.92     14 NaN   5.0
30435  30436  163000   55.58      5 NaN  16.0
30838  30839   66000   16.50      9 NaN   5.0

[120 rows x 6 columns]


In [53]:
# 所在のNaNを埋める
train_data_2 = train_data_1
train_data_2.loc[train_data_2["所在"].isnull(), "所在"] = train_data_2["階層"]

In [62]:
# 埋めた後の確認
print(train_data_2[train_data_2["id"]==30839][["所在", "階層"]])

        所在   階層
30838  5.0  5.0


In [85]:
# IDの削除
train_data_2_no_ID = train_data_2.drop(["id"], axis=1)

### テストデータ

In [65]:
# 埋める前の確認
print(test_data_1[test_data_1["階層"].isnull()])
print(test_data_1[test_data_1["所在"].isnull()])

         id   面積m2  間取りID  所在  階層
9204  40675  94.91     21 NaN NaN
          id   面積m2  間取りID  所在   階層
125    31596  68.28     21 NaN  2.0
229    31700  34.18      1 NaN  3.0
262    31733  60.06     10 NaN  7.0
835    32306  47.30      5 NaN  7.0
1076   32547  54.63     10 NaN  3.0
...      ...    ...    ...  ..  ...
30529  62000  40.73      5 NaN  3.0
30781  62252  58.51     14 NaN  2.0
30826  62297  46.26     19 NaN  2.0
31073  62544  16.01      9 NaN  3.0
31184  62655  16.62      3 NaN  2.0

[114 rows x 5 columns]


In [66]:
# 所在のNaNを埋める
test_data_2 = test_data_1
test_data_2.loc[test_data_2["所在"].isnull(), "所在"] = test_data_2["階層"]

In [68]:
print(test_data_2[test_data_2["所在"].isnull()])

         id   面積m2  間取りID  所在  階層
9204  40675  94.91     21 NaN NaN


In [78]:
# 所在も階層も空欄のデータは間取りと面積の近いデータで埋める
print(test_data_1[(test_data_1["間取りID"] == 21) & (test_data_1["面積m2"] > 90) & (test_data_1["面積m2"] < 95)].mean())

id       44448.676471
面積m2        92.323824
間取りID       21.000000
所在           6.181818
階層           9.878788
dtype: float64


In [79]:
test_data_2.loc[test_data_2["id"]==40675, "所在"] = float(6)
test_data_2.loc[test_data_2["id"]==40675, "階層"] = float(9)

In [82]:
# 埋めた後の確認
print(test_data_2[test_data_2["id"]==40675][["所在", "階層"]])

       所在   階層
9204  6.0  9.0


In [86]:
# IDの削除
test_data_2_no_ID = test_data_2.drop(["id"], axis=1)

## データチェック2

In [87]:
train_data_2.isnull().sum()

id       0
賃料       0
面積m2     0
間取りID    0
所在       0
階層       0
dtype: int64

In [88]:
test_data_2.isnull().sum()

id       0
面積m2     0
間取りID    0
所在       0
階層       0
dtype: int64

## 学習 

In [89]:
# 特徴データと目的変数の設定
train_x = train_data_2_no_ID.drop(["賃料"], axis=1)
train_y = train_data_2_no_ID["賃料"]

test_x = test_data_2_no_ID

In [90]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

scores = []
# GBDT用のハイパーパラメータ
params = {"objective": "reg:squarederror", "sileng":1, "random_state":71}
num_round = 50

# クロスバリデーション
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx, in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    # GBDTで学習実行
    dtrain = xgb.DMatrix(tr_x, label=tr_y) # enable_categorical=True
    dvalid = xgb.DMatrix(va_x, label=va_y)
    dtest = xgb.DMatrix(test_x)
    
    watchlist = [(dtrain, "train"), (dvalid, "eval")]
    model = xgb.train(params, dtrain, num_round, evals=watchlist)

[21:22:25] WARNING: ..\src\learner.cc:541: 
Parameters: { sileng } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:101465.04688	eval-rmse:101138.06250
[1]	train-rmse:75652.92969	eval-rmse:75961.78125
[2]	train-rmse:58471.25781	eval-rmse:59686.94141
[3]	train-rmse:47379.08984	eval-rmse:49409.09766
[4]	train-rmse:40410.04688	eval-rmse:43076.93750
[5]	train-rmse:36206.45703	eval-rmse:39191.50000
[6]	train-rmse:33817.42578	eval-rmse:37214.18750
[7]	train-rmse:32345.94336	eval-rmse:35972.23047
[8]	train-rmse:31395.93359	eval-rmse:35343.58594
[9]	train-rmse:30863.94336	eval-rmse:34922.28906
[10]	train-rmse:30505.58203	eval-rmse:34721.75781
[11]	train-rmse:30226.54883	eval-rmse:34567.12891
[12]	train-rmse:29897.07031	eval-rmse:34501.45312
[13]	train-rmse:29747.70117	eval

## 予測

In [92]:
# テストデータでの予測
pred = model.predict(dtest)

In [93]:
pred

array([159249.5  ,  79441.29 ,  95082.625, ..., 176337.58 , 250241.25 ,
        85037.41 ], dtype=float32)

In [94]:
len(pred)

31262

In [95]:
len(test_data_1)

31262

## データ出力

In [96]:
# 予測結果の結合
test_data["予測結果"] = pred

In [97]:
# 予測結果をint型に変換
test_data["予測結果"] = test_data["予測結果"].astype(int)

In [98]:
result = test_data[["id", "予測結果"]]

In [99]:
result

,id,予測結果
0,31471,159249
1,31472,79441
2,31473,95082
3,31474,68753
4,31475,139176
...,...,...
31257,62728,102943
31258,62729,106165
31259,62730,176337
31260,62731,250241


In [100]:
result.to_csv("result.csv", header=False, index=False)